<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 8.5 - Prompting Large Language Models

In this lab we will practise prompting with a few Large Language Models (LLMs) using Groq (not to be confused with Grok). Groq is a platform that provides access to their custom-built AI hardware via APIs, allowing users to run open-source models such as Llama.

We shall see that while LLMs are powerful tools, how you ask a question or frame a task can dramatically influence the results obtained.

## Set-up

Step 1: Sign up for a free Groq account at https://console.groq.com/home .

Step 2: Create a new API key at https://console.groq.com/keys. Copy-paste it into an empty text file called 'groq_key.txt'.

Running the next cell will then read in this key and assign it to the variable `groq_key`.

In [2]:
pip install groq


Note: you may need to restart the kernel to use updated packages.


In [3]:
groqfilename = 'groq_key.txt'

try:
    with open(groqfilename, 'r') as f:
        groq_key = f.read().strip()
except FileNotFoundError:
    print(f"'{groqfilename}' file not found")


In [4]:
from groq import Groq
import requests
import pandas as pd
from IPython.display import Markdown

In [5]:
#!pip install groq

First create an instance of the Groq client:

In [7]:
client = Groq(api_key=groq_key)

The following code shows what models are currently accessible through Groq. `context_window` refers to the size of memory (in tokens) during a session and `max_completion_tokens` is the maximum number of tokens that are generated in an output.

In [14]:
print(repr(groq_key))


'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'


In [19]:
url = "https://api.groq.com/openai/v1/models"

headers = {
    "Authorization": f"Bearer {groq_key}",
    "Content-Type": "application/json"
}

response = requests.get(url, headers=headers)

pd.DataFrame(response.json()['data']).sort_values(['created'], ascending=False)


,id,object,created,owned_by,active,context_window,public_apps,max_completion_tokens
14,meta-llama/llama-prompt-guard-2-86m,model,1748632165,Meta,True,512,None,512
19,meta-llama/llama-prompt-guard-2-22m,model,1748632101,Meta,True,512,None,512
4,qwen/qwen3-32b,model,1748396646,Alibaba Cloud,True,131072,None,40960
11,meta-llama/llama-guard-4-12b,model,1746743847,Meta,True,131072,None,1024
9,meta-llama/llama-4-maverick-17b-128e-instruct,model,1743877158,Meta,True,131072,None,8192
1,meta-llama/llama-4-scout-17b-16e-instruct,model,1743874824,Meta,True,131072,None,8192
15,compound-beta-mini,model,1742953279,Groq,True,131072,None,8192
16,qwen-qwq-32b,model,1741214760,Alibaba Cloud,True,131072,None,131072
20,compound-beta,model,1740880017,Groq,True,131072,None,8192
17,playai-tts-arabic,model,1740682783,PlayAI,True,8192,None,8192


The Groq client object enables interaction with the Groq REST API and a chat completion request is made via the client.chat.completions.create method.

The most important arguments of the client.chat.completions.create method are the following:
* messages: a list of messages (dictionary form) that make up the conversation to date
* model: a string indicating which model to use (see [list of models](https://console.groq.com/docs/models))
* max_completion_tokens: the maximum number of tokens that are generated in the chat completion
* response_format: setting this to `{ "type": "json_object" }` enables JSON output
* seed: sample deterministically as best as possible, though identical outputs each time are not guaranteed
* temperature: between 0 and 2 where higher values like 0.8 make the output more random (creative) and values like 0.2 are more focused and deterministic


In [23]:
help(client.chat.completions.create)

Help on method create in module groq.resources.chat.completions:

create(*, messages: 'Iterable[ChatCompletionMessageParam]', model: "Union[str, Literal['gemma2-9b-it', 'llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'llama-guard-3-8b', 'llama3-70b-8192', 'llama3-8b-8192']]", exclude_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, frequency_penalty: 'Optional[float] | NotGiven' = NOT_GIVEN, function_call: 'Optional[completion_create_params.FunctionCall] | NotGiven' = NOT_GIVEN, functions: 'Optional[Iterable[completion_create_params.Function]] | NotGiven' = NOT_GIVEN, include_domains: 'Optional[List[str]] | NotGiven' = NOT_GIVEN, logit_bias: 'Optional[Dict[str, int]] | NotGiven' = NOT_GIVEN, logprobs: 'Optional[bool] | NotGiven' = NOT_GIVEN, max_completion_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, max_tokens: 'Optional[int] | NotGiven' = NOT_GIVEN, metadata: 'Optional[Dict[str, str]] | NotGiven' = NOT_GIVEN, n: 'Optional[int] | NotGiven' = NOT_GIVEN, parallel_tool_calls:

As a first example, note how the messages input is given as a list of a dictionaries with `role` and `content` keys. This is in a ChatML format recognised by many LLMs.

In [25]:
chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system", # sets the persona of the model
            "content": "You are a helpful assistant."
        },
        {
            "role": "user", # what the user wants the assistant to do
            "content": "Explain briefly how large language models work",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: The models are trained on massive datasets of text from various sources, such as books, articles, and conversations.
2. **Learning patterns and relationships**: Through this training, the models learn to recognize patterns and relationships between words, phrases, and ideas in language.
3. **Using neural networks**: The models use complex neural networks to analyze and generate text, allowing them to predict the next word or character in a sequence.
4. **Generating text based on context**: When given a prompt or input, the model uses its learned patterns and relationships to generate text that is coherent and contextually relevant.

This enables large language models to perform tasks like answering questions, translating text, and even creating original content, such as stories or dialogue.


The output is in Markdown format so the following line formats this text.

In [27]:
Markdown(chat_completion.choices[0].message.content)

Large language models are artificial intelligence (AI) systems that process and generate human-like language. They work by:

1. **Training on vast amounts of text data**: The models are trained on massive datasets of text from various sources, such as books, articles, and conversations.
2. **Learning patterns and relationships**: Through this training, the models learn to recognize patterns and relationships between words, phrases, and ideas in language.
3. **Using neural networks**: The models use complex neural networks to analyze and generate text, allowing them to predict the next word or character in a sequence.
4. **Generating text based on context**: When given a prompt or input, the model uses its learned patterns and relationships to generate text that is coherent and contextually relevant.

This enables large language models to perform tasks like answering questions, translating text, and even creating original content, such as stories or dialogue.

## Text summarisation

We start with a llama3-8b-8192, a model using just over 8 billion parameters with at most 8192 tokens produced as output.

Here is an article to be summarised from the [cnn_dailymail](https://huggingface.co/datasets/cnn_dailymail) dataset:

In [29]:
story = """
SAN FRANCISCO, California (CNN) -- A magnitude 4.2 earthquake shook the San Francisco area Friday at 4:42 a.m. PT (7:42 a.m. ET), the U.S. Geological Survey reported. The quake left about 2,000 customers without power, said David Eisenhower, a spokesman for Pacific Gas and Light. Under the USGS classification, a magnitude 4.2 earthquake is considered "light," which it says usually causes minimal damage. "We had quite a spike in calls, mostly calls of inquiry, none of any injury, none of any damage that was reported," said Capt. Al Casciato of the San Francisco police. "It was fairly mild." Watch police describe concerned calls immediately after the quake » . The quake was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles, the USGS said. Oakland is just east of San Francisco, across San Francisco Bay. An Oakland police dispatcher told CNN the quake set off alarms at people's homes. The shaking lasted about 50 seconds, said CNN meteorologist Chad Myers. According to the USGS, magnitude 4.2 quakes are felt indoors and may break dishes and windows and overturn unstable objects. Pendulum clocks may stop.
"""

**Exercise:**
Summarise the story text using the following three prompts. Use the format given above but here there is no need to set the persona (i.e. only include one dictionary in the messages list when calling `client.chat.completions.create`.) Comment on any differences.

1) "Summarise the following article in 3 sentences."

2) "Give me a TL;DR of this text."

3) "What's the key takeaway here?"

In [32]:
prompts = ["Summarise the following article in 3 sentences. ", "Give me a TL;DR of this text. ", "What's the key takeaway here?"]
#content will be p + story for p in prompts

# ANSWER
for p in prompts:
    response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": p + story}]
)

    print(p, '\n', response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a 3 sentence summary of the article:

A magnitude 4.2 earthquake struck the San Francisco area early Friday morning, causing minimal damage and prompting about 2,000 power outages. The quake, which was centered about two miles east-northeast of Oakland, was described as "light" and left no reported injuries or significant damage. The shaking lasted about 50 seconds and triggered alarms at people's homes and may have caused objects to fall or break, as is typical for an earthquake of this magnitude.
Give me a TL;DR of this text.  
 Here is a TL;DR of the text:

A magnitude 4.2 earthquake struck the San Francisco area on Friday at 4:42am, leaving around 2,000 customers without power. The USGS classifies the quake as "light" and says it caused minimal damage, with no reported injuries or significant damage. The quake was centered near Oakland and lasted about 50 seconds.
What's the key takeaway here? 
 The key takeaway is that a m

Run the above code again below and note that the answers may differ. This is due to the probabilistic nature of LLM token generation.

In [35]:
# ANSWER
for p in prompts:
    response = client.chat.completions.create(
                model="llama3-8b-8192",
                messages=[{"role": "user", "content": p + story}]
)

    print(p, '\n', response.choices[0].message.content)

Summarise the following article in 3 sentences.  
 Here is a summary of the article in 3 sentences:

A magnitude 4.2 earthquake struck the San Francisco area at 4:42 a.m. PT on Friday, causing minimal damage and leaving about 2,000 customers without power. The USGS classified the quake as "light" and reported that it was centered about two miles east-northeast of Oakland, at a depth of 3.6 miles. The shaking lasted about 50 seconds and caused some alarms to go off at people's homes, with no reports of injury or significant damage.
Give me a TL;DR of this text.  
 Here is a brief summary of the text:

A magnitude 4.2 earthquake struck the San Francisco area early Friday morning, causing about 2,000 customers to lose power. The quake was considered "light" and caused minimal damage, with no reports of injuries or significant damage. The quake was centered near Oakland and lasted for about 50 seconds, with some people reporting their home alarms went off.
What's the key takeaway here? 
 T

## Text completion

**Exercise**: In this section adjust the `max_completion_tokens` and `temperature` settings below to obtain different responses. Show some examples with the prompt "Continue the story: It was a great time to be alive" with the model "llama-3.1-8b-instant".

* max_completion_tokens - the maximum number of tokens to generate. Note that longer words are made of multiple tokens (set to 200 and 500)
* temperature (positive number) - the higher the number the more random (creative) the output (set to 0.2, 0.8, 2)

In [38]:
# ANSWER (set max_completion_tokens=200, do not have a temperature setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens=200,
)

print(response.choices[0].message.content)


It was a great time to be alive, and Emily couldn't help but feel a sense of excitement and possibility as she walked through the crowded streets of the city. The year was 1969, and the music scene was booming. The air was electric with the sounds of Jimi Hendrix, The Beatles, and Janis Joplin.

Emily, a 20-year-old art student, was standing on the corner of 23rd Street and Broadway, surrounded by the pulsating rhythm of the city. She had just left a lively conversation with her friend, Sarah, about the latest happenings in the art world. They had spent hours discussing the latest movements - Abstract Expressionism, Pop Art, and the emergence of Conceptual Art.

As Emily gazed out at the sea of faces passing by, she noticed a flyer taped to a streetlamp. It was an advertisement for the latest exhibit at the Museum of Modern Art (MoMA), featuring works by Warhol, Lichtenstein


In [40]:
# ANSWER (set max_completion_tokens=500, do not have a temperature setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    max_completion_tokens=500,
)

print(response.choices[0].message.content)

It was a great time to be alive. The summer sun was shining brightly, casting a warm glow over the vibrant city. The air was alive with the hum of music and the chatter of people from all walks of life. everywhere, neon lights seemed to dance in every possible color, and the smell of street food wafted through the air, teasing the senses.

Lena, a young artist, was sitting on a bench, strumming a melancholy tune on her guitar. She had always found solace in music, and the rhythm seemed to match the beat of her own heart. As she played, she couldn't help but think about the world around her. It was a time of great change, a time of great uncertainty.

But amidst all the chaos, there was a sense of hope. A sense that anything was possible, that the future was bright and full of promise. Lena's eyes met those of a stranger, a boy with piercing blue eyes and a mop of messy brown hair. He was walking by, his feet tapping out the beat of the music to Lena's guitar.

For a moment, they locked

In [42]:
# ANSWER (set temperature = 0.2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive"}],
    temperature = 0.2,
)

print(response.choices[0].message.content)


It was a great time to be alive, and for Emily, it was especially true. The year was 1969, and the world was on the cusp of a revolution. Music, fashion, and art were all pushing the boundaries of what was considered acceptable, and Emily was right in the middle of it all.

She had just turned 20, and her life was a whirlwind of concerts, protests, and late-night conversations about the meaning of life. She had just started dating a free-spirited artist named Max, who had a passion for painting and a penchant for getting into trouble.

Together, they spent their days exploring the city, attending underground music shows, and talking about the latest issues of the day. The Vietnam War was raging, and the civil rights movement was gaining momentum. Emily and Max were both passionate about social justice, and they spent countless hours discussing the latest news and developments.

One night, as they were walking home from a concert, they stumbled upon a group of people gathered in the par

In [45]:
# ANSWER (set temperature = 1, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive."}],
    temperature = 1,
)

print(response.choices[0].message.content)

The sun was shining bright, casting a warm glow over the bustling streets of the city. People of all ages walked hand in hand, their faces filled with joy and their hearts full of hope. It was a great time to be alive, indeed.

Lena, a young and ambitious journalist, sat at her desk, tapping away at her computer's keyboard as she worked on her latest article. She had just received a tip about a groundbreaking new development in the city's downtown area, and she was determined to get the exclusive scoop.

As she typed away, the sounds of the city drifted in through the window: the chatter of pedestrians, the honking of cars, and the wail of sirens in the distance. Lena's editor, Rachel, poked her head into the room, a look of excitement on her face.

"Lena, I have some news," Rachel said, barely containing her enthusiasm. "I just got word that the city council is going to announce a major new project tonight at a press conference."

Lena's eyes lit up. "What kind of project?" she asked,

Note what happens when the temperature is set too high!

In [48]:
# ANSWER (set temperature = 2, do not have a max_completion_tokens setting)
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Continue the story: It was a great time to be alive."}],
    temperature = 2,
)

print(response.choices[0].message.content)

It was the end of the 20th century, and the world seemed to be filled with endless possibilities. The cold war had just ended, technological advancements were revolutionizing every facet of life, and a sense of hope and idealism hung in the air. For Emily, a brilliant and vibrant college student, life was just beginning.

She had just turned 25, and along with her friends, Sarah and Rachel, they had embarked on a once-in-a-lifetime journey – a road trip across the American West. The car was loaded with snacks, maps, and their trusty music player. As they cruised down the California Highway in their battered-but-beloved Volkswagen Beetle named Bertha, Emily couldn't contain her excitement.

"You know, Rach, I always wanted to write something epic." Emily ex claimed, her eyes locked onto miles of golden desert stretching out across their windshield. Rach raised an brow and replied 'what you had in mind?'

"I was thinking, what we just experienced? It deserves to be written – not like som

### Zero-shot and one-short prompting for question-answering

This section shows the impact of prompting on the response. Zero-shot prompting means we provide the prompt without any examples or additional context. Let us initially ask Mistral a question using no prompting.

In [50]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

Chemical reactions occur when two or more substances, called reactants, interact with each other to form new substances, called products. The interaction between the reactants can result in a physical change, such as a change in state (e.g., solid to liquid), or a chemical change, such as the formation of a new compound.

Here are the general steps involved in a chemical reaction:

1. **Collision**: The reactant molecules collide with each other, resulting in the formation of a temporary complex.
2. **Reaction**: The reactant molecules interact with each other, leading to the breaking or forming of chemical bonds.
3. **Formation of products**: The reactant molecules are converted into new products, which can be in the form of atoms, molecules, or ions.
4. **Release of energy**: The reaction may release or absorb energy, depending on the type of reaction.

Types of chemical reactions:

1. **Synthesis reaction**: Two or more reactants combine to form a new compound. (e.g., 2H2 + O2 → 2H2O)
2. **Decomposition reaction**: A single reactant breaks down into two or more products. (e.g., 2H2O → 2H2 + O2)
3. **Single displacement reaction**: One element is displaced by another element in a compound. (e.g., Zn + CuSO4 → ZnSO4 + Cu)
4. **Double displacement reaction**: Two compounds exchange partners to form two new compounds. (e.g., NaCl + AgNO3 → NaNO3 + AgCl)
5. **Combustion reaction**: A substance reacts with oxygen to release heat and light. (e.g., CH4 + 2O2 → CO2 + 2H2O)
6. **Redox reaction**: A reaction that involves the transfer of electrons between reactants. (e.g., Zn + CuSO4 → ZnSO4 + Cu)

To predict how two chemicals react, you need to consider the following factors:

1. **Chemical properties**: The properties of the reactants, such as their reactivity, electronegativity, and bond strength.
2. **Chemical structure**: The molecular structure of the reactants, including the types of bonds and functional groups present.
3. **Reaction conditions**: The temperature, pressure, and concentration of the reactants.
4. **Catalysts**: The presence of catalysts, which can speed up or slow down the reaction.

By considering these factors, you can predict the type of reaction that will occur between two chemicals and the products that will be formed.

**Exercise:** Ask the same question but modify the prompt to return the answer to the same question in a simpler form (still using the llama-3.1-8b-instant model). Experiment with different prompts.

In [53]:
# ANSWER
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": "Answer the following question as though I am 10 years old. How do two chemicals react?"}],
    temperature = 0.8,
)

Markdown(response.choices[0].message.content)

Imagine you have a big box of LEGO blocks in different colors. Each color represents a different chemical. Now, let's say you have two LEGO blocks that are different colors, like red and blue.

When you mix these two LEGO blocks together, something cool happens. The colors mix and create a new color, like purple. This is kind of like what happens when two chemicals react.

Chemicals are like the LEGO blocks, and when they mix together, they change and create something new. Sometimes, this new thing can be useful, like creating medicine to help us feel better. Other times, it might not be so useful and can even be bad for us.

When two chemicals react, they usually do a few things:

1. They mix together and change into a new substance.
2. They might make heat, like when you mix baking soda and vinegar and see a fizzing reaction.
3. They might make light, like when you mix two special chemicals that glow in the dark.

So, chemical reactions are like LEGO block mixing, but instead of making toys, they make new substances that can be really cool or really important.

### One-shot prompting ###

Next, note the dramatic change when we give the following template setting a new role and providing an English question followed by a French translation.

In [56]:
response = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "system",
             "content": "You translate English to French."},
              {"role": "user",
               "content": "What time is it?"},
               {"role": "assistant",
               "content": "Quelle heure est-il?"},
              {"role": "user",
               "content": "How do two chemicals react?"}],
    temperature = 0.8,
)
print(response.choices[0].message.content)

Comment deux composés chimiques réagissent-ils?


### Few-shot prompting

Recall that since the text generation process outputs one token at a time, their outputs often need adjusting. This is where examples can help.

In [58]:
prompt1 = "I'm gonna head out now, see you later."
response1 = "I will be leaving now. See you later."

prompt2 =  "That movie was super cool!"
response2 = "The movie was very impressive."

prompt3 = "Can't make it to the meeting, sorry."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a professional editor. Rewrite casual sentences into a formal tone."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


I regret to inform you that I will be unable to attend the meeting.


The output can also be moulded to provide SQL output.

In [60]:
prompt1 = "Show me all users who signed up in the last 30 days."
response1 = "SELECT * FROM users WHERE signup_date >= CURRENT_DATE - INTERVAL '30 days';"

prompt2 = "What is the average order value?"
response2 =  "SELECT AVG(order_total) FROM orders;"

prompt3 = "List products that are out of stock."

response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are an assistant that translates natural language to SQL."},
        {"role": "user", "content": prompt1},
        {"role": "assistant", "content": response1},
        {"role": "user", "content": prompt2},
        {"role": "assistant", "content": response2},
        {"role": "user", "content": prompt3},
    ]
)

print(response.choices[0].message.content.strip())


SELECT * FROM products WHERE stock_quantity = 0;


**Exercise**: Create a few examples to train the "llama3-70b-8192" LLM to take in user content in the form below and provide output as a pandas dataframe. Use the `exec` function to execute its output to display the answer of sample input as a data frame.

Example:

given the user content

"""

| col1 | col2 | col3

| 32 | 27 | 25

| 64 | 23 | 14

"""

train the model to output

df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})



In [63]:
#ANSWER
user1 = """col1 | col2 | col3
32 | 27 | 25
64 | 23 | 14
"""

output1 = """
df = pd.DataFrame({'col1': [32, 64], 'col2': [27, 23], 'col3': [25, 14]})
"""

user2 = """col1 | col2
23 | 12
8 | 76
7 | 5
"""
output2 = """
df = pd.DataFrame({'col1': [23, 8, 7], 'col2': [12, 76, 5]})
"""
user3 = """colA | colB | colC
23 | 12 | 54
8 | 76 | 32
7 | 5 | 3
"""


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are a data scientist who will receive data input as a string and provide output as a pandas dataframe called df. Use the examples to guide you"},
        {"role": "user", "content": user1},
        {"role": "assistant", "content": output1},
        {"role": "user", "content": user2},
        {"role": "user", "content": output2},
        {"role": "user", "content": user3}
    ]
)

exec(response.choices[0].message.content.strip()) # string executed as Python code
df


,colA,colB,colC
0,23,12,54
1,8,76,32
2,7,5,3


Also show what happens when the question is asked in the absence of a system role and without few-shot prompting.

In [66]:
# ANSWER
response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "user", "content": user3}
    ]
)
response.choices[0].message.content.strip()

"It looks like you've provided a table with three columns: colA, colB, and colC. Here's a formatted version of the table:\n\n| colA | colB | colC |\n| --- | --- | --- |\n| 23 | 12 | 54 |\n| 8 | 76 | 32 |\n| 7 | 5 | 3 |\n\nWhat would you like to do with this table?"

### Chain-of-thought prompting

The results of question-answering can also be improved by prompting the LLM to provide intermediate steps.

**Exercise**: Using the following prompts, compare the answers of the "llama3-8b-8192" model (set seed=21). (If this model is no longer available choose a model with relatively few parameters.)

zero_shot_prompt = "How many s's are in the word 'success'?"

chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

In [69]:
# ANSWER
zero_shot_prompt = "How many s's are in the word 'success'?"
chain_of_thought_prompt = "How many s's are in the word 'success'? Explain your answer step by step by going through each letter in turn."

response1 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": zero_shot_prompt}],
    seed = 21
)

response2 = client.chat.completions.create(
    model="llama-3.1-8b-instant",
    messages=[{"role": "user", "content": chain_of_thought_prompt}],
    seed = 21
)

print('------zero-shot-prompt------')
print(response1.choices[0].message.content)

print('------chain-of-thought------')
print(response2.choices[0].message.content)

------zero-shot-prompt------
There are 2 s's in the word 'success'.
------chain-of-thought------
To count the number of 's's in the word "success", we'll go through each letter one by one:

1. S
   There's one 's' in the first position.

2. U
   Nothing else is changing here as this is just a 'U'.

3. C
   We still have the same number of s's, 1.

4. C
   The 's' number remains 1.

5. E
   Still at 1 s'.

6. S
   There's now a second 's'.

7. S
   There's now a third 's'

In the end, there are 3 's's in the word "success".


## Comparison of LLMs

**Exercise**: Compare the performance of 2 LLMs by outputting the answers of the following questions into a dataframe.

    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",

Column headings:

Model Name | Question | Answer

In [72]:
pd.set_option('display.max_colwidth', None) # allows wide dataframes to be viewed
models = ["gemma2-9b-it", "llama-3.1-8b-instant"] #can edit this

# ANSWER
prompts = [
    "Tell me a joke about data science.",
    "How can one calculate 22 * 13 mentally?",
    "Write a creative story about a baby learning to crawl.",
]

results = {'Model Name': [], 'Question': [], 'Answer': []}

for model in models:
    for prompt in prompts:
        results['Model Name'].append(model)
        results['Question'].append(prompt)
        try:
            output = client.chat.completions.create(model = model, messages=[{"role": "user", "content": prompt}])
            results['Answer'].append(output.choices[0].message.content.strip())

        except Exception as e:
            print(f"Error with {model}: {e}")
            results['Answer'].append((prompt, "ERROR"))


df = pd.DataFrame(results)
df

,Model Name,Question,Answer
0,gemma2-9b-it,Tell me a joke about data science.,Why did the data scientist break up with the statistician? \n\nBecause they had too many significant differences! 😂 \n\n\nLet me know if you want to hear another one! 😄
1,gemma2-9b-it,How can one calculate 22 * 13 mentally?,"Here's a trick to calculate 22 * 13 mentally:\n\n**Break it down:**\n\n* **Think of 22 as (20 + 2)**\n* **Multiply each part by 13:**\n * 20 * 13 = 260\n * 2 * 13 = 26\n\n* **Add the two results:** 260 + 26 = 286\n\n\nTherefore, 22 * 13 = 286"
2,gemma2-9b-it,Write a creative story about a baby learning to crawl.,"Bertie, a fuzzy-headed cherub with eyes like melted chocolate, gazed longingly at the towering figure of his teddy bear, Mr. Snugglesworth. Mr. Snugglesworth sat atop his playmat, a smug, cotton smile stitched onto his furry face. Bertie wanted to reach him, desperately.\n\nHis attempts were...exasperating. He’d flail his knobby legs, kick a wayward foot across the room, and roll with the fury of a tiny hurricane. He could scoot, he could turn, he could even sit up with a wobbly grin and clap his chubby hands, but Mr. Snugglesworth remained frustratingly out of reach.\n\nThen, one sunny morning, something shifted. Bertie, inspired by his reflection in the shiny mobile above him, decided to try something new. He saw his chubby arms reach forward, mirroring the jerky motion of the dangling baubles.\n\nHe pushed himself up, his fingers digging into the plush carpet. The room tilted, his vision blurring for a moment, but he steadied himself. His legs wobbled, but he held on. He took a tentative step, a leg inching forward, then another. His grin widened, transforming his face into a permanent open-mouthed expression.\n\nHe moved like a wobbling tank, propelled by sheer determination. Every inch gained was a victory. He bumped into his toys, fell on his face, rolled onto his back, and giggled uncontrollably.\n\nHe was crawling, even if it was more like an ungainly ballet performed by a fuzzy, leggy disaster. But it was crawling, nonetheless.\n\nBertie, spurred on by his accomplishment, crawled towards his nemesis: Mr. Snugglesworth. Each inch closer was met with squeals of delight. Finally, his tiny hands reached the bear’s soft fur.\n\nHe collapsed on top of Mr. Snugglesworth, burying his face in the bear's cotton cheek. Mr. Snugglesworth, silent in his plush dignity, seemed to wink his sewn-on eye. Bertie had conquered the world, or at least, his playmat – and for a baby, that was enough.\n\n\nHe’d learned that day, the most valuable lesson a baby can learn: even the most daunting mountains can be climbed, one wobbly step at a time."
3,llama-3.1-8b-instant,Tell me a joke about data science.,Why did the data scientist bring a ladder to the party?\n\nBecause they heard the drinks were on the house.
4,llama-3.1-8b-instant,How can one calculate 22 * 13 mentally?,"To calculate 22 * 13 mentally, you can use the following method:\n\n1. Break down the numbers into smaller parts. In this case, 13 can be broken down into 10 + 3.\n2. Calculate 22 * 10: 220 (since 20 * 10 = 200 and adding 2 to it gives you 220)\n3. Now, calculate 22 * 3: 66\n4. Add the two results: 220 + 66 = 286\n\nSo, the answer is 286.\n\nAnother method is:\n\n1. Multiply 22 by a nearby round number, 20, which gives you 400, and multiply by 2 (as in 22).\n2. Calculate 22 x 0.1 * 3 = 6.6\n3. Then add 6.6 to 400 to get your solution which is 400 + 66 = 286"
5,llama-3.1-8b-instant,Write a creative story about a baby learning to crawl.,"Once upon a time, in a cozy little home surrounded by lush green plants and warm sunshine, there lived a baby named Luna. Luna was a bundle of energy and curiosity, always eager to explore the world around her. Her bright blue eyes sparkled with excitement as she discovered new sights, sounds, and smells every day.\n\nAs Luna grew bigger and stronger, she began to feel the urge to move beyond the confines of her crib. She watched with en

### Bonus

See if you can prompt an LLM to perform sentiment analysis (output 'Positive' or 'Negative' only) on a given piece of text.

In [75]:
# ANSWER
input1 = "I absolutely loved the way the story unfolded."
output1 = "Positive"

input2 = "The food was cold and completely flavorless."
output2 = "Negative"

input3 = "She handled the situation with grace and professionalism."


response = client.chat.completions.create(
    model="llama3-70b-8192",
    messages=[
        {"role": "system", "content": "You are amazing at sentiment analysis. Give the sentiment of the next sentence as the examples show."},
        {"role": "user", "content": input1},
        {"role": "assistant", "content": output1},
        {"role": "user", "content": input2},
        {"role": "assistant", "content": output2},
        {"role": "user", "content": input3},
    ]
)
response.choices[0].message.content

'Positive'

## Conclusion

We worked with a few Large Language Models (LLMs) using Groq and experimented with prompting for summarisation, text completion and question-answering tasks.

We also explored controlling the randomness (creativity) of output through the temperature setting and tried different types of prompting to achieve desired forms of output.

## References
1. [Groq's prompting guide](https://console.groq.com/docs/prompting)
2. [Groq's playground](https://console.groq.com/playground)



---



---



> > > > > > > > > © 2025 Institute of Data


---



---



